In [1]:
# Dependencies
import numpy as np
import pandas as pd

In [2]:
unfilteredData = pd.read_csv('Resources/WMT.csv')
unfilteredData.head()

,date,open,high,low,close,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime
0,2014-01-27,64.7650,64.9747,64.5029,64.7825,9105139,9105139,-0.235889,-0.363,64.7739,"Jan 27, 14",0.000000
1,2014-01-28,64.8786,65.8746,64.7388,65.2368,6035231,6035231,0.454305,0.701,65.3045,"Jan 28, 14",0.007013
2,2014-01-29,65.7785,65.8484,64.7126,64.7388,8440854,8440854,-0.497990,-0.763,61.0517,"Jan 29, 14",-0.000675
3,2014-01-30,65.1232,65.6037,64.9660,65.3067,6742046,6742046,0.567883,0.877,65.2975,"Jan 30, 14",0.008092
4,2014-01-31,64.5816,65.6911,64.3369,65.2455,10665285,10665285,-0.061155,-0.094,65.3223,"Jan 31, 14",0.007147


In [3]:
unfilteredData = unfilteredData.drop(columns=["volume", "unadjustedVolume", "change", "changePercent", "vwap", "label", "changeOverTime"])
unfilteredData.head()

,date,open,high,low,close
0,2014-01-27,64.7650,64.9747,64.5029,64.7825
1,2014-01-28,64.8786,65.8746,64.7388,65.2368
2,2014-01-29,65.7785,65.8484,64.7126,64.7388
3,2014-01-30,65.1232,65.6037,64.9660,65.3067
4,2014-01-31,64.5816,65.6911,64.3369,65.2455


In [4]:
lastIndex = len(unfilteredData)
print(lastIndex)

1259


In [5]:
startDate = unfilteredData.loc[unfilteredData["date"] == "2016-06-01"]
startDateIndex = startDate.index.tolist()
print(startDateIndex[0])

591


In [6]:
filteredData = unfilteredData.loc[startDateIndex[0]:lastIndex]

In [7]:
filteredData.head()

,date,open,high,low,close
591,2016-06-01,65.8547,66.2330,65.7706,65.8547
592,2016-06-02,65.8547,66.3217,65.5231,66.2750
593,2016-06-03,65.9201,66.3684,65.8733,66.2003
594,2016-06-06,66.8075,66.8355,66.3217,66.3684
595,2016-06-07,66.4151,66.6113,66.1816,66.3497


In [8]:
filteredData.reset_index(inplace=True, drop=True)
filteredData.head()

,date,open,high,low,close
0,2016-06-01,65.8547,66.2330,65.7706,65.8547
1,2016-06-02,65.8547,66.3217,65.5231,66.2750
2,2016-06-03,65.9201,66.3684,65.8733,66.2003
3,2016-06-06,66.8075,66.8355,66.3217,66.3684
4,2016-06-07,66.4151,66.6113,66.1816,66.3497
